# EBH model of episodic memory

# Imports

In [1]:
%matplotlib widget
import numpy as np
import matplotlib.pyplot as plt
import time
import pickle
import math
import sparse_weights as sw
import etile as et

# Initialize

In [2]:
# Start things up
np.random.seed(1)
nodetype = 'int16'

# read in stack of letters
# read in stack of letters
font = 'HN'
n_pix = 100
n_let = 26
xjitter = 30
yjitter = 15
density = 15 # in thousandths
#with open('letters/letterstack_j{:d}_f{:d}.pkl'.format(jitter,fontsize), 'rb') as f:
with open('letters/letterstack_f{:s}_jx{:d}_jy{:d}_d{:d}.pkl'.format(font,xjitter,yjitter,density), 'rb') as f:
    newlist = pickle.load(f)
    letterstack = newlist[0].astype(nodetype)
    # symbols: forward, shift, lowercase, lower, querycase, upper
    symbols = newlist[1].astype(nodetype)
lettermat = np.reshape(letterstack,(2*n_let,n_pix*n_pix))

# Initialize network
n_nodes = n_pix*n_pix
n_keys = n_let+1 # letters plus initial condition
weight_density = 0.2
weight_std = 0.0001
learning_rate = 0.025
activation_threshold = 0.5

weights = sw.sparse_weights(n_nodes,density=weight_density,weight_std=weight_std)

def stepfunc(a,thresh):
    return (a>thresh).astype(nodetype)


# Go

In [3]:
# Go
training_record = np.zeros((n_let,n_pix,n_pix),dtype=nodetype)
test_record = np.zeros((n_let,n_pix,n_pix),dtype=nodetype)

# Train
nodes = lettermat[0,:].copy()
for iletter in range(n_let):
    if iletter:
        old_nodes = nodes.copy()
        nodes = lettermat[iletter,:]
        weights.data += learning_rate * nodes[weights.row] * old_nodes[weights.col]
    training_record[iletter,:,:] = np.reshape(nodes,(n_pix,n_pix))    
    
# Test
nodes = lettermat[0,:].copy()
for iletter in range(n_let):
    test_record[iletter,:,:] = np.reshape(nodes,(n_pix,n_pix))
    nodes = stepfunc(weights.dot(nodes),activation_threshold)


fig,ax = plt.subplots(2, 1, figsize=(8,10))
ax[0].imshow(et.etile(np.pad(training_record,((0,4),(0,0),(0,0)),'constant')))
ax[1].imshow(et.etile(np.pad(test_record,((0,4),(0,0),(0,0)),'constant')))
cormat = lettermat @ lettermat.T
#im=ax[2].imshow(cormat)
#fig.colorbar(im,ax=ax[2])
plt.show()

FigureCanvasNbAgg()

In [4]:
plt.savefig('episodic.png')